<a href="https://colab.research.google.com/github/3dsf/ArtLineFFMPEG/blob/main/ArtLineBufferFFMPEG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ArtLine Video Conversion
implementation of 
https://github.com/vijishmadhavan/ArtLine



# Setup


## Install Libs

In [11]:
!apt install ffmpeg
!pip install youtube-dl fastai==1.0.61

^C


In [12]:
import os
os.makedirs("input", exist_ok=True)
os.makedirs("output", exist_ok=True)

## Download Infernce Code

In [13]:
!wget -O run.py https://gist.githubusercontent.com/3dsf/5827842bd6dc4fe7ada5dda868eb4599/raw/6ffc0b5010e30a9e52af684f752adb2142718961/run.py

--2021-01-06 03:53:17--  https://gist.githubusercontent.com/3dsf/5827842bd6dc4fe7ada5dda868eb4599/raw/6ffc0b5010e30a9e52af684f752adb2142718961/run.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3931 (3.8K) [text/plain]
Saving to: ‘run.py’

run.py              100%[===================>]   3.84K  --.-KB/s    in 0s      

2021-01-06 03:53:17 (64.7 MB/s) - ‘run.py’ saved [3931/3931]



# ---> Select your video to process

In [4]:
youtubeVideo = "https://www.youtube.com/watch?v=bSvEwnYI2Lc" #@param {type:"string"}
!rm input.mp4
!time(youtube-dl -f '[ext=mp4]' --output "input.%(ext)s"  $youtubeVideo)

rm: cannot remove 'input.mp4': No such file or directory
[youtube] bSvEwnYI2Lc: Downloading webpage
[youtube] bSvEwnYI2Lc: Downloading MPD manifest
[download] Destination: input.mp4
[download] 100% of 11.54MiB in 00:00

real	0m1.847s
user	0m0.694s
sys	0m0.065s


# Decode Video

In [5]:
#Audio
!rm audio.mp3
!time(ffmpeg -i input.mp4  audio.mp3 -y) 

rm: cannot remove 'audio.mp3': No such file or directory
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enab

In [6]:
#Visual
!rm input/*.bmp output/*.bmp
!time(ffmpeg -i input.mp4 input/frame.%05d.bmp)

rm: cannot remove 'input/*.bmp': No such file or directory
rm: cannot remove 'output/*.bmp': No such file or directory
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorb

# ---> Select the Model to Use
If you **didn't connect** to your **drive** and copy the models, **automatic fallback** is provided. 

In [7]:
artLineModel = "ArtLine_1024.pkl" #@param ["ArtLine_500.pkl", "ArtLine_650.pkl", "ArtLine_1024.pkl"]
pathToModel = os.path.join("/content/drive/",artLineModel)

downloadModel = {
    "ArtLine_500.pkl": "https://www.dropbox.com/s/p9lynpwygjmeed2/ArtLine_500.pkl",
    "ArtLine_650.pkl": "https://www.dropbox.com/s/starqc9qd2e1lg1/ArtLine_650.pkl",
    "ArtLine_1024.pkl": "https://www.dropbox.com/s/rq90q9lr9arwdp8/ArtLine_1024%20%281%29.pkl"
}
import os.path as path
import subprocess
if path.isfile(pathToModel) == False : 
    if path.isfile(artLineModel) == False :
        import subprocess
        print("Downloading")
        subprocess.call("wget -O " + artLineModel + " " + downloadModel[artLineModel] , shell=True)
        pathToModel = artLineModel
    else :
        print("Found Local Version")
        pathToModel = artLineModel

Downloading


# Run Model

In [8]:
!time(python run.py $pathToModel)


MODEL FOUND : ArtLine_1024.pkl
INFERENCE DEVICE : cuda
MODEL LOADED IN : 4.822887053
not sure the time tracking is accurate in colab
input/frame.03275.bmp (1/3357) time : 0.4360825530000003
input/frame.03229.bmp (2/3357) time : 0.3377642180000002
input/frame.01715.bmp (3/3357) time : 0.3353896289999998
input/frame.03235.bmp (4/3357) time : 0.3395883140000002
input/frame.02276.bmp (5/3357) time : 0.33996363600000024
input/frame.00006.bmp (6/3357) time : 0.341073669
input/frame.01058.bmp (7/3357) time : 0.3372965679999993
input/frame.01776.bmp (8/3357) time : 0.3365954379999998
input/frame.01292.bmp (9/3357) time : 0.3393860069999999
input/frame.03163.bmp (10/3357) time : 0.33565714099999866
input/frame.03161.bmp (11/3357) time : 0.3388017889999997
input/frame.02504.bmp (12/3357) time : 0.3397655809999982
input/frame.02431.bmp (13/3357) time : 0.34030847100000017
input/frame.01117.bmp (14/3357) time : 0.3383256800000005
input/frame.01736.bmp (15/3357) time : 0.33838597599999964
input/fr

# ---> Select the Output Name and **Encode**


In [9]:
output_name = "nomke.heycat.webm" #@param {type:"string"}
!time(ffmpeg -framerate `ffmpeg -i input.mp4 2>&1 | sed -n "s/.*, \(.*\) fp.*/\1/p"` -thread_queue_size 8192 -i output/frame.%05d.bmp   -i audio.mp3  $output_name -y)

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# Download result

In [10]:
from google.colab import files

files.download(output_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>